In [25]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# -----------------------------
# Step 1: Load and preprocess dataset
# -----------------------------
def load_data(path="recipes_data.csv"):
    df = pd.read_csv(path)

    # Check if 'protein', 'carbs', 'fat' columns exist
    if all(col in df.columns for col in ['protein', 'carbs', 'fat']):
        scaler = MinMaxScaler()
        df[['protein', 'carbs', 'fat']] = scaler.fit_transform(df[['protein', 'carbs', 'fat']])
    else:
        # Generate placeholder values (can be improved later with inference)
        df['protein'] = np.random.rand(len(df))
        df['carbs'] = np.random.rand(len(df))
        df['fat'] = np.random.rand(len(df))
        scaler = None  # Avoid referencing undefined variable

    return df, scaler

# -----------------------------
# Step 2: Ingredient inference functions
# -----------------------------
NOTVEGETARIAN_INGREDIENTS = [
    'chicken', 'beef', 'pork', 'lamb', 'turkey', 'duck',
    'bacon', 'ham', 'sausage', 'veal', 'goat', 'prosciutto',
    'salami', 'pepperoni', 'ground beef', 'ground turkey',
    'corned beef', 'pastrami', 'ox', 'venison', 'rabbit',
    'bison', 'quail', 'goose', 'chorizo', 'kielbasa',
    'bratwurst', 'meatballs', 'liver', 'gizzard', 'tripe'
]

GLUTEN_INGREDIENTS = [
    'wheat', 'barley', 'rye', 'triticale', 'malt', 'brewer’s yeast',
    'farro', 'spelt', 'kamut', 'semolina', 'durum', 'graham flour',
    'einkorn', 'emmer', 'bulgur', 'couscous', 'seitan',
    'pasta', 'bread', 'breadcrumbs', 'crackers', 'cereal',
    'cakes', 'cookies', 'pies', 'pastries', 'flour tortillas',
    'soy sauce', 'miso', 'beer', 'ale', 'lager'
]


HIGH_PROTEIN_INGREDIENTS = ['chicken', 'beef', 'tofu', 'lentils', 'fish', 'tempeh', 'quinoa', 'eggs']
HIGH_CARB_INGREDIENTS = ['pasta', 'bread', 'rice', 'potatoes', 'corn']
HIGH_FAT_INGREDIENTS = ['avocado', 'olive oil', 'butter', 'nuts', 'seeds']

# -----------------------------
# Step 3: Infer attributes based on ingredients
# -----------------------------
def infer_attributes(ingredients):
    vegetarian = not any(ing in ingredients.lower() for ing in NOTVEGETARIAN_INGREDIENTS)
    gluten_free = not any(ing in ingredients.lower() for ing in GLUTEN_INGREDIENTS)
    
    protein_score = sum(1 for ing in HIGH_PROTEIN_INGREDIENTS if ing in ingredients.lower())
    carb_score = sum(1 for ing in HIGH_CARB_INGREDIENTS if ing in ingredients.lower())
    fat_score = sum(1 for ing in HIGH_FAT_INGREDIENTS if ing in ingredients.lower())
    
    return vegetarian, gluten_free, protein_score, carb_score, fat_score

# -----------------------------
# Step 4: Filter recipes based on inferred preferences
# -----------------------------
def filter_recipes(df, preferences):
    filtered_recipes = []
    for _, row in df.iterrows():
        ingredients = row['ingredients']
        
        vegetarian, gluten_free, protein_score, carb_score, fat_score = infer_attributes(ingredients)
        
        if preferences.get('vegetarian') and not vegetarian:
            continue
        if preferences.get('gluten_free') and not gluten_free:
            continue
        if preferences.get('preferred_ingredients') and not any(ing in ingredients.lower() for ing in preferences['preferred_ingredients']):
            continue
        
        row['vegetarian'] = vegetarian
        row['gluten_free'] = gluten_free
        row['protein'] = protein_score
        row['carbs'] = carb_score
        row['fat'] = fat_score
        
        filtered_recipes.append(row)
    
    return pd.DataFrame(filtered_recipes)

# -----------------------------
# Step 5: Create target scores for meal preferences
# -----------------------------
def compute_targets(df, preferences):
    targets = []
    for _, row in df.iterrows():
        score = 0
        if preferences.get('high_protein'):
            score += row['protein']
        if preferences.get('high_carbs'):
            score += row['carbs']
        if preferences.get('high_fat'):
            score += row['fat']
        targets.append(score)
    targets = np.array(targets)
    targets = (targets - targets.min()) / (targets.max() - targets.min() + 1e-6)
    return targets

# -----------------------------
# Step 6: Define and train the model
# -----------------------------
def build_and_train_model(X, y):
    model = Sequential([
        Dense(16, activation='relu', input_shape=(3,)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.01), loss='mse')
    model.fit(X, y, epochs=50, verbose=0)
    return model

# -----------------------------
# Step 7: Generate meal plan
# -----------------------------
def generate_meal_plan(df, model):
    X = df[['protein', 'carbs', 'fat']].values
    predictions = model.predict(X).flatten()
    df['score'] = predictions

    top_recipes = df.sort_values(by='score', ascending=False).head(15)
    top_recipes = top_recipes.sample(frac=1).reset_index(drop=True)

    meal_plan = []
    for day in range(5):
        day_meals = top_recipes.iloc[day*3:(day+1)*3]
        meal_plan.append(day_meals[['title', 'ingredients', 'directions']].to_dict(orient='records'))
    return meal_plan


In [26]:
# -----------------------------
# Main Function Call
# -----------------------------
user_preferences = {
    'vegetarian': False,
    'gluten_free': True,
    'preferred_ingredients': ['tofu', 'chickpeas', 'broccoli'],
    'high_protein': False,
    'high_carbs': True,
    'high_fat': False
}

df, scaler = load_data("recipes_data.csv")
df = filter_recipes(df, user_preferences)
if len(df) < 15:
    print("Not enough recipes after filtering.")
else:
    X = df[['protein', 'carbs', 'fat']].values
    y = compute_targets(df, user_preferences)

    model = build_and_train_model(X, y)
    


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
meal_plan = generate_meal_plan(df, model)

for day_idx, day in enumerate(meal_plan):
    print(f"\nDay {day_idx + 1}:")
    for meal_idx, meal in enumerate(day):
        print(f"  Meal {meal_idx + 1}: {meal['title']} {meal['ingredients']}")

1487/1487 ━━━━━━━━━━━━━━━━━━━━ 0s 206us/step

Day 1:
  Meal 1: Creamy Golden Potato-Squash Soup ["1 medium acorn or golden acorn squash, about 1 pound", "1 tablespoon olive oil", "1 large onion, chopped", "2 to 3 cloves garlic, minced", "4 medium-large potatoes (about 1 1/2 pounds), peeled and diced", "2 bay leaves", "2 vegetable bouillon cubes", "1/2 teaspoon good-quality curry powder", "1/2 teaspoon dried dill", "One 12.3-ounce package firm silken tofu, coarsely crumbled", "2 cups rice milk, or as needed", "1/4 cup Silk creamer, optional", "Salt and freshly ground pepper to taste", "Minced fresh parsley for garnish, optional"]
  Meal 2: Hearty Vegetable Curry ["3 tablespoons olive oil", "1 large onion, sliced", "3 medium sweet potatoes, peeled & cubed", "1 large head cauliflower, cut in flourettes", "4 teaspoons curry powder", "1 (15 ounce) can vegetable broth", "2 tablespoons cornstarch", "1 (15 ounce) can diced tomatoes", "1 (15 ounce) can chickpeas, drained & rinsed", "1 1/2 teasp